# Create a planar mesh for use with k223d
This notebook will:
- show you how to generate a planar mesh
- save in vtk format
- run k223d code

In [19]:
# systems operations
import sys
import os

# numerical operations 
# import numpy as np


# for running k223d
import subprocess

# track time of k223d
import time

# # for meshing 
# import gmsh
import meshio

#plotting mesh 
# import plotly.express as px
# import plotly.graph_objs as go


### Parameters

In [2]:
kilo = 1000

### Custom Python modules

In [3]:
folder_path = '../local_py_scripts'
sys.path.append(folder_path)
from plotting import *
from notebook_utils import *
from fault_scaling import *

## Fault parameters

In [4]:
Mw = 7
target_area = mag2area(Mw,"Strasser")

Fwidth = 20*kilo
print('Width of fault(km)...:',Fwidth/kilo) 

Flength =  target_area/Fwidth # 30*kilo
print('Length of fault based on width and scaling relationship(km)...:',Flength/kilo) 
Fdip = 30.0*np.pi/180.

lcar = 1.*kilo  # resolution on the fault 
rupt_vel = 2000. # constant rupture velocity in m/s  
# rupt_grad = 0.25   # m/s


# nucleation of fault is at the centre of the fault 
nuc_y = Flength/2
nuc_x = Fwidth*np.cos(Fdip)/2



Width of fault(km)...: 20.0
Length of fault based on width and scaling relationship(km)...: 77.08502264747791


# Mesh fault using Gmsh

In [4]:
def mesh_planar_fault(Flength,Fwidth,Fdip,lcar,file_name):
    gmsh.initialize()
    gmsh.clear() 
# add points delineating the top (fault trace) and bottom of the fault 
    gmsh.model.geo.addPoint( 0, 0, 0, lcar, 0)
    gmsh.model.geo.addPoint( 0, Flength, 0, lcar, 1)
    gmsh.model.geo.addPoint( Fwidth*np.cos(Fdip), Flength, -Fwidth*np.sin(Fdip), lcar, 2)
    gmsh.model.geo.addPoint( Fwidth*np.cos(Fdip), 0, -Fwidth*np.sin(Fdip), lcar, 3)
# joint the points together to turn the fault into segments that will be meshed
    gmsh.model.geo.addLine(0, 1, 0)
    gmsh.model.geo.addLine(1, 2, 1)
    gmsh.model.geo.addLine(2, 3, 2)
    gmsh.model.geo.addLine(3, 0, 3)
# curve loop
    gmsh.model.geo.addCurveLoop([0, 1, 2, 3], 1)
    gmsh.model.geo.addPlaneSurface([1], 1)

    gmsh.model.geo.synchronize()
    gmsh.model.mesh.generate(3)   # put as 3 as we are generating a 3d mesh

# write results to file 
    gmsh.write(file_name+".vtk")   
    gmsh.write(file_name+".msh")      
    
# Finalize Gmsh
    gmsh.finalize()       

In [5]:
mesh_planar_fault(Flength,Fwidth,Fdip,lcar,'planar_mesh')

Info    : Clearing all models and views...
Info    : Done clearing all models and views
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 0 (Line)
Info    : [ 30%] Meshing curve 1 (Line)
Info    : [ 50%] Meshing curve 2 (Line)
Info    : [ 80%] Meshing curve 3 (Line)
Info    : Done meshing 1D (Wall 0.00149117s, CPU 0.000564s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.0687707s, CPU 0.060103s)
Info    : Meshing 3D...
Info    : Done meshing 3D (Wall 0.000412917s, CPU 4.4e-05s)
Info    : 1948 nodes 3898 elements
Info    : Writing 'planar_mesh.vtk'...
Info    : Done writing 'planar_mesh.vtk'
Info    : Writing 'planar_mesh.msh'...
Info    : Done writing 'planar_mesh.msh'


## Extract mesh and add additional fields

In [6]:
x,y,z,cells = extract_mesh('planar_mesh.msh')

ncells = len(cells)
cell_type = np.zeros(ncells,dtype=int)
cell_vel = np.zeros(ncells,dtype=float)

Info    : Reading 'planar_mesh.msh'...
Info    : 9 entities
Info    : 1948 nodes
Info    : 3898 elements
Info    : Done reading 'planar_mesh.msh'
number of nodes      1948
number of elements     3698


In [ ]:
points = np.transpose([x,y,z])
plot_mesh(cells,points)

## Add velocity to mesh

In [8]:
# set velocity for each cell to a constant value 

cell_vel = np.zeros(ncells)
cell_vel[:] = rupt_vel
# for i in range(n_ncells):
#     c_id = new_cells[i]
#     # use the mean depth of each cell
#     mean_depth = np.abs(np.mean(new_z[c_id]))   
#     cell_vel[i] = rupt_vel  + rupt_grad*(mean_depth) 

In [ ]:
plot_mesh_cell(cells,points,cell_vel,'velocity')

### Assign initial travel time 
To decide where to pick for the earthquake we plot with plotly


In [10]:
itime = assign_nucleation_location(nuc_x,nuc_y,x,y,z,cells,cell_vel)


Source is in cell 1980
Initial travel time on starting cell edges: [0.39926891 0.16292887 0.27075039]


In [ ]:
plot_mesh_node(cells,points,itime,'initial time')

## Asign surface rupture 

In [12]:
print('fault depth range (m):    ',np.min(z),np.max(z))

fault depth range (m):     -9999.999999999998 0.0


In [13]:
# set surface rupture depth 
surface_depth = 0.0 
nnodes = len(z)
surface_nodes =  np.zeros(nnodes,dtype=int)
for i in range(nnodes):
    if z[i] >= surface_depth:
        surface_nodes[i] = 1
print('number of surface nodes.....',np.sum(surface_nodes))

number of surface nodes..... 79


In [ ]:
plot_highlight_nodes(cells,points,surface_nodes,'surface nodes')


# Output mesh for k223d

In [15]:
mesh_file = 'input.vtk'
write_vtk(mesh_file,points,cells,cell_vel,itime,surface_nodes)

# Prepare input file 

Make changes to inputfile 
specify : 
1) earthquake size,
2) name of the file containing the mesh
3) name of the output file  


In [20]:
file_path = 'input_file'

updates = {
    'magnitude':Mw,
    'mesh_file':mesh_file,
    'out_file':'prova_planar'
}

update_inputfile(file_path, updates)

## Run k223d

In [21]:
# call k223d and print elapsed time
start=time.time()
subprocess.call(["../../code/k223d.x"])
end=time.time()
print(end-start)

 In single use mode
 enter read_vtk_mesh
 No. of nodes....        1948
 no. of cells........         3698
 number of nucleation nodes found...           3
         365         367         368
 nucleation cell id:             1981
 fault  surface found
 no. of surface nodes:               79
 exit read_vtk_mesh
 starting allocation
 create FVsNodes
 start EToE
 starting trilateration.....
 exiting based on fast protocol ....
6.082401990890503


## Read in Results and plot

In [22]:
res = meshio.read("prova_planar.vtk")
tri = res.get_cells_type("triangle")
# X = res.points[:,0];Y = res.points[:,1];Z = res.points[:,2]

a = np.array(res.cell_data['vel'])
res_vel = a.flatten('F')

a = np.array(res.cell_data['slip'])
res_slip = a.flatten('F')

a = np.array(res.point_data['Rupt_time'])
res_time = a.flatten('F')

In [ ]:
plot_mesh_node(tri,res.points,res_time,'Rupture Time (s)')

In [ ]:
plot_mesh_cell(tri,res.points,res_slip,'slip (m)')